In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

import TheoreticallyOptimalStrategy as TOS
from marketsimcode import compute_portvals, compute_stats
import indicators
from strategy_evaluation.ManualStrategy import ManualStrategy
from strategy_evaluation.StrategyLearner import StrategyLearner

In [4]:
symbol = "JPM"
in_sample_sd = dt.datetime(2008, 1, 1)
in_sample_ed = dt.datetime(2009, 12, 31)
out_sample_sd = dt.datetime(2010, 1, 1)
out_sample_ed = dt.datetime(2011, 12, 31)
sv = 100000
commission = 9.95
impact = 0.005

#MANUAL STRATEGY

In [5]:
# Manual Strategy

sd = out_sample_sd
ed = out_sample_ed
indicator = indicators.Indicators(symbol, pd.date_range(sd, ed))
bbp = indicator.get_bbp(window = 20)
rsi = indicator.get_rsi(window = 14)
macd = indicator.get_macd().loc[sd:] # My version of indicators.py is also returning values 9 market days prior to sd

# Calculate MACD crossover
macd_prev = macd.shift(1)
macd_prev.iloc[0] = macd_prev.iloc[1]
conditions = [
    (macd_prev <= 0) & (macd > 0),
    (macd_prev >= 0) & (macd < 0)
]
values = [1, -1]
# Documentation: https://numpy.org/doc/2.1/reference/generated/numpy.select.html
macd_crossover = pd.DataFrame({'result': np.select(conditions, values, default=0)}, index=macd.index, columns=["result"])

# Generate signals
signals_cond = [
    ((bbp <= 0) & (rsi <= 30)) | ((bbp <= 0) & (macd_crossover["result"] == 1)) | ((rsi <= 30) & (macd_crossover["result"] == 1)),
    ((bbp >= 100) & (rsi >= 70)) | ((bbp >= 100) & (macd_crossover["result"] == 1)) | ((rsi >= 70) & (macd_crossover["result"] == 1))
]
signal_values = [1, -1]
signals = pd.DataFrame({'result': np.select(signals_cond, signal_values, default=0)}, index=bbp.index)

# Based on indicators on T day, we will place orders on T+1 day
signals = signals.shift(1)
signals = signals.fillna(0)
signals.drop(signals.index[-1])

# Generate Manual Strategy trades
net_position = 0
manual_trades = pd.DataFrame(0, index=bbp.index, columns=[symbol])

for date in signals.index:
    if signals.loc[date].iloc[0] == 1:
        manual_trades.loc[date] = 1000 - net_position
    elif signals.loc[date].iloc[0] == -1:
        manual_trades.loc[date] = -1000 - net_position

    net_position += manual_trades.loc[pd.to_datetime(date), symbol]

# portvals = compute_portvals(manual_trades, start_val=sv, commission=commission, impact=impact)
portvals = compute_portvals(manual_trades, start_val=sv, commission=commission, impact=impact)

print(compute_stats(portvals))

# manual_trades
portvals

(0.12344300000000086, 0.00026066868511829747, 0.007720292661427542, 0.5359883716721906)


2010-01-04    100000.00
2010-01-05    100000.00
2010-01-06    100000.00
2010-01-07     99776.45
2010-01-08     99876.45
                ...    
2011-12-23    112654.30
2011-12-27    112124.30
2011-12-28    111754.30
2011-12-29    112504.30
2011-12-30    112344.30
Length: 504, dtype: float64

In [36]:
compute_stats(portvals)

(-0.570665, -0.0016337761780937103, 0.05524187128897186, -0.46948794792624565)

#STRATEGY LEARNER

In [2]:
symbol = "JPM"
in_sample_sd = dt.datetime(2008, 1, 1)
in_sample_ed = dt.datetime(2009, 12, 31)
out_sample_sd = dt.datetime(2010, 1, 1)
out_sample_ed = dt.datetime(2011, 12, 31)
sv = 100000
commission = 0
impact = 0

manualstrategy = ManualStrategy()
strategylearner = StrategyLearner(impact=0.005, commission=9.95)
# trades = manualstrategy.testpolicy(symbol="JPM", sd=in_sample_sd, ed=in_sample_ed)
strategylearner.add_evidence(symbol="JPM", sd=in_sample_sd, ed=in_sample_ed)
trades = strategylearner.testPolicy(symbol="JPM", sd=in_sample_sd, ed=in_sample_ed)

portvals = compute_portvals(trades, start_val=sv, commission=commission, impact=impact)
# portvals = compute_portvals(trades, start_val=sv, commission=commission, impact=impact)

print(compute_stats(portvals))
portvals


EPOCH:  0

Reward:  -0.0

Scores & Count:  [-0.0] 0



EPOCH:  1

Reward:  0.0

Scores & Count:  [-0.0, 0.0] 1



EPOCH:  2

Reward:  0.0

Scores & Count:  [-0.0, 0.0, 0.0] 2



EPOCH:  3

Reward:  0.0

Scores & Count:  [-0.0, 0.0, 0.0, 0.0] 3



EPOCH:  4

Reward:  0.0

Scores & Count:  [-0.0, 0.0, 0.0, 0.0, 0.0] 4




/Users/sankalpphadnis/Documents/OMSCS/ML4T/ML4T Spring 2025 Setup/strategy_evaluation/StrategyLearner.py:181: RuntimeWarning: invalid value encountered in double_scalars
  if len(scores) >= 3 and (abs(scores[-2]/scores[-1] - 1) <= 0.01) and (abs(scores[-3]/scores[-2] - 1) <= 0.01):



EPOCH:  5

Reward:  0.0

Scores & Count:  [-0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 5



EPOCH:  6

Reward:  0.0

Scores & Count:  [-0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 6



EPOCH:  7

Reward:  0.0

Scores & Count:  [-0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 7



EPOCH:  8

Reward:  0.0

Scores & Count:  [-0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 8



EPOCH:  9

Reward:  0.0

Scores & Count:  [-0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 9



EPOCH:  10

Reward:  0.0

Scores & Count:  [-0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 10



EPOCH:  11

Reward:  0.0

Scores & Count:  [-0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 11



EPOCH:  12

Reward:  0.0

Scores & Count:  [-0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 12



EPOCH:  13

Reward:  0.0

Scores & Count:  [-0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 13



EPOCH:  14

Reward:  0.0

Scores & Count:  [-0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

2008-01-02    100000.0
2008-01-03    100000.0
2008-01-04     99130.0
2008-01-07     99510.0
2008-01-08     98010.0
                ...   
2009-12-24    283290.0
2009-12-28    283130.0
2009-12-29    282910.0
2009-12-30    282870.0
2009-12-31    282740.0
Length: 505, dtype: float64